# Modelo SVM para predecir la presión (SVR)

In [1]:
from pprint import pprint
from typing import Union, Tuple
from pathlib import Path

import numpy as np
import pandas as pd

# Importar los clasificadores y escaladores necesarios
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator
from sklearn.svm import SVR
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler

# Importar GridSearchCV para realizar la búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

## Definir función para buscar hiperparámetros

In [2]:
# definimos la función que toma datos y devuelve los mejores parámetros

def search_params(
        estimator: BaseEstimator,
        params: dict,
        X: np.ndarray, 
        y: np.ndarray, 
        random: bool = False, 
        n_iter: int = 100,
    ) -> Tuple[SVR, dict]:
    # Definir los parámetros para GridSearchCV

    print(f"Los parámetros para probar en nuestras pipelines son:")
    pprint(params)

    # Creamos un pipeline con parámetros por defecto (serán sustituidos)
    pipeline = Pipeline([
        ('scaler', None),
        ('regressor', estimator())
    ])

    if random:
        search = RandomizedSearchCV(pipeline, params, cv=5, n_jobs=-1, 
                                    scoring='neg_mean_squared_error', verbose=1, n_iter=n_iter)
    else:
        search = GridSearchCV(pipeline, params, cv=5, n_jobs=-1, 
                              scoring='neg_mean_squared_error', verbose=1)

    search.fit(X, y)

    # Obtener los mejores hiperparámetros y el mejor modelo
    best_params = search.best_params_
    best_model = search.best_estimator_

    print(f"Los mejores parámetros de la busqueda: {best_params}")
    print(f"El mejor modelo de la busqueda: {best_model}")

    return best_model, best_params

## Cargar el DataFrame

In [3]:
# Definir la ruta de los datos
data_path: Path = Path(r"../data/processed")
if not data_path.exists():
    print(f"El path {data_path} no existe, favor de revisar.")

# Imprimir archivos en la carpeta de datos
pprint(list(data_path.glob("*")))

[PosixPath('../data/processed/datos_molienda_prueba.parquet'),
 PosixPath('../data/processed/df_model_no_outliers.parquet'),
 PosixPath('../data/processed/df_model_tidy.parquet')]


In [4]:
parquet_path = data_path / "df_model_tidy.parquet"

df = pd.read_parquet(parquet_path)
df.head(3)

,date,velocidad,potencia,rendimiento,ruido,p80,f80,per_solidos,wi,spi,imp_criticos,imp_estandares,jb,presion
0,2022-11-15 00:07:50,75.761029,39281.348296,65.653747,22.932414,64.260113,36.574719,71.583049,94.815049,54.50695,7.101110,65.598165,9.183127,87.199926
1,2022-11-15 00:08:00,75.761029,39850.320731,65.691489,22.932414,64.260113,36.555098,71.579590,94.815049,54.50695,7.191338,65.798205,9.180693,87.402467
2,2022-11-15 00:08:10,75.761029,39761.319456,65.783858,21.329909,64.260113,36.535478,71.585002,94.815049,54.50695,7.281566,65.998244,9.178803,87.492231


## Definir `X` y `y`

In [5]:
X: np.ndarray = df.drop(columns=["presion", "date"]).to_numpy()
y: np.ndarray = df["presion"].to_numpy()

# Verificar tipos
print(f"{type(X) = }")
print(f"{type(y) = }")

# Verificar formas
print(f"{X.shape = }")
print(f"{y.shape = }")

type(X) = <class 'numpy.ndarray'>
type(y) = <class 'numpy.ndarray'>
X.shape = (248618, 12)
y.shape = (248618,)


# Reservar una pequeña parte para probar modelo

In [12]:
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
# Ya que el entrenamiento es muy tardado, tomemos solo una parte de los datos
train_size = 0.10
X_train_tune, _, y_train_tune, _ = train_test_split(X, y, train_size=train_size)

print(f"Se usarán {X_train_tune.shape[0]:,} entradas para el entrenamiento del modelo usando 5 folds)")
print(f"Se reservó un {int(test_size*100)}% de los datos ({X_test.shape[0]:,}) para validar al final del entrenamiento.")

Se usarán 24,861 entradas para el entrenamiento del modelo usando 5 folds)
Se reservó un 10% de los datos (24,862) para validar al final del entrenamiento.


## Buscar hiperparámetros

In [8]:
# NOTA: para los parámetros del regresor se utiliza el prefijo regressor__ por convención
# para indicar que ese parámetro es para el valor de la clave 'regressor'
parameters = [
    {
        'scaler': [Normalizer(), StandardScaler(), MinMaxScaler()],
        # Parámetros del clasificador
        'regressor__C': np.linspace(1, 100, 10),
        'regressor__kernel': ['linear', 'rbf', 'poly'],
        'regressor__gamma': ['scale', 'auto'],
    },
]

best_model, best_params = search_params(
    SVR, parameters, X_train_tune, y_train_tune, n_iter=1, random=True,
)

Los parámetros para probar en nuestras pipelines son:
[{'regressor__C': array([  1.,  12.,  23.,  34.,  45.,  56.,  67.,  78.,  89., 100.]),
  'regressor__gamma': ['scale', 'auto'],
  'regressor__kernel': ['linear', 'rbf', 'poly'],
  'scaler': [Normalizer(), StandardScaler(), MinMaxScaler()]}]
Fitting 5 folds for each of 1 candidates, totalling 5 fits


Los mejores parámetros de la busqueda: {'scaler': MinMaxScaler(), 'regressor__kernel': 'poly', 'regressor__gamma': 'auto', 'regressor__C': 45.0}
El mejor modelo de la busqueda: Pipeline(steps=[('scaler', MinMaxScaler()),
                ('regressor', SVR(C=45.0, gamma='auto', kernel='poly'))])


In [35]:
best_model.fit(X_train, y_train)

dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])

In [10]:
print('Training set score: ' + str(best_model.score(X_train, y_train)))
print('Test set score: ' + str(best_model.score(X_test, y_test)))

Training set score: 0.722488668671342
Test set score: 0.7249967510527712


## Guardar el modelo


In [13]:
import joblib

# Guardar el mejor modelo en un archivo
model_filename = '../models/best_svr.pkl'
joblib.dump(best_model, model_filename)
print(f"Modelo guardado en {model_filename}")

Modelo guardado en ../models/best_svr.pkl


In [14]:
# Cargar el modelo desde el archivo
loaded_model = joblib.load(model_filename)
print(f"Modelo cargado: {loaded_model}")

Modelo cargado: Pipeline(steps=[('scaler', MinMaxScaler()),
                ('regressor', SVR(C=45.0, gamma='auto', kernel='poly'))])
